In [1]:
# for rewards, store n, min, max, list, list of estimated delivery dates, intn shipping
# for creator, get n backed
#
# create new databases -- rewards, creators
# start with tech, then for all entries in my database 

In [3]:
# Features I already have _______________________________________
# - title word/char len
# - blurb word/char len
# - launch time (in Unix time)
# - project duration
# - goal

In [57]:
# Other possible features to extract _____________________________
# - n pictures -- use beautiful soup
# - has video
# - project body length 
# - project n misspelligns
# - text quality, sentiment analysis
# lower priority -- 
# - international projects


In [5]:
import requests
from bs4 import BeautifulSoup
import pprint
import pymysql as mdb
import re
import pandas as pd
import numpy as np
from unidecode import unidecode
import json

In [29]:
from extract_kickstarter_features import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
con = mdb.connect('localhost','root','hobbes','kickstarter')

In [11]:
with con:
    
    cur = con.cursor()
    cur.execute("SHOW TABLES")
    
    rows = cur.fetchall()
    for row in rows:
        print row

('Project_info',)
('Rewards',)
('Scraped_pages',)


In [58]:
# Create and describe my new table
with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS Rewards")
    cur.execute("CREATE TABLE Rewards(Id INT PRIMARY KEY,\
                 N_rewards INT,\
                 Min_reward INT,\
                 Max_reward BIGINT,\
                 Reward_list VARCHAR(1500),\
                 Reward_expected VARCHAR(1500),\
                 Ships_intn TINYINT)")
    
    cur.execute("DESCRIBE Rewards")
    rows = cur.fetchall()
    
    for row in rows:
        print row[0:3]
    cur.close()

('Id', 'int(11)', 'NO')
('N_rewards', 'int(11)', 'YES')
('Min_reward', 'int(11)', 'YES')
('Max_reward', 'bigint(20)', 'YES')
('Reward_list', 'varchar(1500)', 'YES')
('Reward_expected', 'varchar(1500)', 'YES')
('Ships_intn', 'tinyint(4)', 'YES')


In [52]:
# Just start with getting features from tech projects

In [14]:
with con:
    cur = con.cursor()
    cur.execute("SELECT Id FROM Project_info \
                WHERE Category_slug LIKE 'tech%'")
    ids = cur.fetchall()
    cur.close()

In [15]:
ids_array = np.asarray(ids)

In [16]:
np.shape(ids_array)

(9351, 1)

In [64]:
print primary_key
print primary_key-ids_array[0]

14732
[7611]


In [63]:
print retrieved_text

()


In [76]:
with con:
    
    for id in ids[7611:]:
        primary_key = id[0]
        
        cur = con.cursor()
        cur.execute("SELECT Project_html from Scraped_pages WHERE Id = %s",primary_key)
        retrieved_text = cur.fetchall()
        cur.close()
    
        this_soup = BeautifulSoup(retrieved_text[0][0])
        project_text = this_soup.get_text()
        rewards_predictors = analyze_rewards(project_text)
        
        #print [primary_key]+rewards_predictors
        
        cur = con.cursor()
        cur.execute("INSERT INTO Rewards () VALUES(%s,%s,%s,%s,%s,%s,%s)",\
                    [primary_key]+rewards_predictors)
        con.commit()
        cur.close()

In [78]:
# # now validate a few
# with con:
#     cur = con.cursor()
#     cur.execute("SELECT * from Rewards WHERE Id = 14732")
#     retrieved_text = cur.fetchall()
#     print retrieved_text[0]
#     cur.close()
# # looks good!

(14732, 8, 10, 500, '[10, 20, 39, 45, 64, 119, 250, 500]', '["Jan 2015", "Jan 2015", "Jan 2015", "Jan 2015", "Jan 2015", "Jan 2015", "Jan 2015", "Jan 2015"]', 0)


In [79]:
# count my rewards
with con:
    cur = con.cursor()
    cur.execute("SELECT COUNT(Id) from Rewards")
    retrieved_text = cur.fetchall()
    print retrieved_text[0]
    cur.close()

(9351,)


In [85]:
# now do this for creator
# Create and describe my new table
with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS Creator")
    cur.execute("CREATE TABLE Creator(Id INT PRIMARY KEY,\
                 N_backed INT)")
    
    cur.execute("DESCRIBE Creator")
    rows = cur.fetchall()
    
    for row in rows:
        print row[0:3]
    cur.close()

('Id', 'int(11)', 'NO')
('N_backed', 'int(11)', 'YES')


In [86]:
np.shape(ids)

(9351, 1)

In [87]:
with con:
    for id in ids:
        primary_key = id[0]
        
        if not primary_key%1000:
            print primary_key
            
        cur = con.cursor()
        cur.execute("SELECT Creator_html from Scraped_pages WHERE Id = %s",primary_key)
        retrieved_text = cur.fetchall()
        cur.close()
    
        this_soup = BeautifulSoup(retrieved_text[0][0])
        creator_text = this_soup.get_text()
        n_backed = get_creator_activity(creator_text)
        
        cur = con.cursor()
        cur.execute("INSERT INTO Creator () VALUES(%s,%s)",\
                    [primary_key,n_backed])
        con.commit()
        cur.close()

8000
9000
10000
11000
12000
13000
14000
15000
16000
